foo.039 Crop Total Nutrient Consumption

http://www.earthstat.org/data-download/
file type: geotiff

In [1]:
# Libraries for downloading data from remote server (may be ftp)
import requests
from urllib.request import urlopen
from contextlib import closing
import shutil

# Library for uploading/downloading data to/from S3
import boto3

# Libraries for handling data
import rasterio as rio
import numpy as np
# from netCDF4 import Dataset
# import pandas as pd
# import scipy

# Libraries for various helper functions
# from datetime import datetime
import os
import threading
import sys
from glob import glob

s3

In [2]:
s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/foo_039_Crop_Total_Nutrient_Consumption/"

s3_file1 = "foo_039_Crop_Total_Nutrient_Consumption_Nitrogen.tif"
s3_file2 = "foo_039_Crop_Total_Nutrient_Consumption_Phosphorus.tif"
s3_file3 = "foo_039_Crop_Total_Nutrient_Consumption_Potassium.tif"

s3_key_orig1 = s3_folder + s3_file1
s3_key_edit1 = s3_key_orig1[0:-4] + "_edit.tif"

s3_key_orig2 = s3_folder + s3_file2
s3_key_edit2 = s3_key_orig2[0:-4] + "_edit.tif"

s3_key_orig3 = s3_folder + s3_file3
s3_key_edit3 = s3_key_orig3[0:-4] + "_edit.tif"

class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

Define local file locations

In [3]:
local_folder = "/Users/Max81007/Desktop/Python/Resource_Watch/Raster/foo.039/FertilizerConsumption_Geotiff/"

file_name1 = "NitrogenFertilizer_TotalConsumption.tif"
file_name2 = "PhosphorusFertilizer_TotalConsumption.tif"
file_name3 = "PotassiumFertilizer_TotalConsumption.tif"

local_orig1 = local_folder + file_name1
local_orig2 = local_folder + file_name2
local_orig3 = local_folder + file_name3

orig_extension_length = 4 #4 for each char in .tif

local_edit1 = local_orig1[:-orig_extension_length] + "edit.tif" 
local_edit2 = local_orig2[:-orig_extension_length] + "edit.tif" 
local_edit3 = local_orig3[:-orig_extension_length] + "edit.tif" 


Use rasterio to reproject and compress

In [4]:
files = [local_orig1, local_orig2, local_orig3]
for file in files:
    with rio.open(file, 'r') as src:
        profile = src.profile
        print(profile)

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 4320, 'height': 2160, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.0833333333333286, 0.0, -180.0,
       0.0, -0.0833333333333286, 89.99999999998977), 'blockxsize': 432, 'blockysize': 224, 'tiled': True, 'compress': 'packbits', 'interleave': 'band'}
{'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 4320, 'height': 2160, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.0833333333333286, 0.0, -180.0,
       0.0, -0.0833333333333286, 89.99999999998977), 'blockxsize': 432, 'blockysize': 224, 'tiled': True, 'compress': 'packbits', 'interleave': 'band'}
{'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 4320, 'height': 2160, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.0833333333333286, 0.0, -180.0,
       0.0, -0.0833333333333286, 89.99999999998977), 'blockxsize': 432, 'blockysize': 224, 'tiled': True, 'compress': 'packbits', 'inte

In [5]:
# Note - this is the core of Vizz's netcdf2tif function
def convert_asc_to_tif(orig_name, edit_name):

    with rio.open(orig_name, 'r') as src:
        # This assumes data is readable by rasterio
        # May need to open instead with netcdf4.Dataset, for example

        data = src.read()[0]

        rows = data.shape[0]
        columns = data.shape[1]

        print(rows)
        print(columns)

        # Latitude bounds
        south_lat = -90
        north_lat = 90

        # Longitude bounds
        west_lon = -180
        east_lon = 180

        transform = rio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, columns, rows)

        # Profile

        no_data_val = None
        target_projection = 'EPSG:4326'
        target_data_type = np.float64

        profile = {
            'driver':'GTiff', 
            'height':rows, 
            'width':columns, 
            'count':1, 
            'dtype':target_data_type, 
            'crs':target_projection, 
            'transform':transform, 
            'compress':'lzw', 
            'nodata': no_data_val
        }

        with rio.open(edit_name, "w", **profile) as dst:
            dst.write(data.astype(profile["dtype"]), 1)

In [6]:
convert_asc_to_tif(local_orig1, local_edit1)
convert_asc_to_tif(local_orig2, local_edit2)
convert_asc_to_tif(local_orig3, local_edit3)


2160
4320
2160
4320
2160
4320


Upload orig and edit files to s3

In [7]:
# Original
s3_upload.upload_file(local_orig1, s3_bucket, s3_key_orig1,
                         Callback=ProgressPercentage(local_orig1))
s3_upload.upload_file(local_orig2, s3_bucket, s3_key_orig2,
                         Callback=ProgressPercentage(local_orig2))
s3_upload.upload_file(local_orig3, s3_bucket, s3_key_orig3,
                         Callback=ProgressPercentage(local_orig3))

# Edit
s3_upload.upload_file(local_edit1, s3_bucket, s3_key_edit1,
                         Callback=ProgressPercentage(local_edit1))
s3_upload.upload_file(local_edit2, s3_bucket, s3_key_edit2,
                         Callback=ProgressPercentage(local_edit2))
s3_upload.upload_file(local_edit3, s3_bucket, s3_key_edit3,
                         Callback=ProgressPercentage(local_edit3))

/Users/Max81007/Desktop/Python/Resource_Watch/Raster/foo.039/FertilizerConsumption_Geotiff/PotassiumFertilizer_TotalConsumptionedit.tif  11188656 / 11188656.0  (100.00%)

Define local file locations for merged files

In [8]:
local_tmp_folder = "/Users/Max81007/Desktop/Python/Resource_Watch/Raster/foo.039/FertilizerConsumption_Geotiff/"

tmp1 = local_tmp_folder + "NitrogenFertilizer_TotalConsumptionedit.tif"
tmp2 = local_tmp_folder + "PhosphorusFertilizer_TotalConsumptionedit.tif"
tmp3 = local_tmp_folder + "PotassiumFertilizer_TotalConsumptionedit.tif"
merge_files = [tmp1, tmp2, tmp3]

tmp_merge = local_tmp_folder + "Foo_039_Fertilizer_TotalConsumption_Merge.tif"

# S3 storage
s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/foo_039_Crop_Total_Nutrient_Consumption/"

s3_file1 = s3_folder + "NitrogenFertilizer_TotalConsumption.tif"
s3_file2 = s3_folder + "PhosphorusFertilizer_TotalConsumption.tif"
s3_file3 = s3_folder + "PotassiumFertilizer_TotalConsumption.tif"

# Make sure these match the order of the merge_files above
s3_files_to_merge = [s3_file1, s3_file2, s3_file3]
band_ids = ["Nitrogen","Phosphorus","Potassium" ]
s3_key_merge = s3_folder + "Foo_039_Fertilizer_TotalConsumption_Merge.tif"

# S3 services
s3_download = boto3.resource("s3")
s3_upload = boto3.client("s3")

# Helper function to view upload progress
class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write(
                    "\r%s  %s / %s  (%.2f%%)" % (
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

In [9]:
with rio.open(merge_files[0]) as src:
    kwargs = src.profile
    
kwargs.update(
    count=len(merge_files)
)
    
with rio.open(tmp_merge, 'w', **kwargs) as dst:
    for idx, file in enumerate(merge_files):
        print(idx)
        with rio.open(file) as src:

            band = idx+1
            windows = src.block_windows()

            for win_id, window in windows:
                src_data = src.read(1, window=window)
                dst.write_band(band, src_data, window=window)

0
1
2


In [10]:
with rio.open(tmp_merge) as src:
    num_bands = src.profile["count"] + 1
    data = {}
    for i in range(1, num_bands):
        data[i] = src.read(i)

In [11]:
files = [tmp_merge]
for file in files:
    with rio.open(file, 'r') as src:
        profile = src.profile
        print(profile)

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 4320, 'height': 2160, 'count': 3, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.08333333333333333, 0.0, -180.0,
       0.0, -0.08333333333333333, 90.0), 'blockxsize': '432', 'blockysize': '224', 'compress': 'lzw', 'interleave': 'band', 'tiled': False}


In [12]:
s3_upload.upload_file(tmp_merge, s3_bucket, s3_key_merge,
           Callback=ProgressPercentage(tmp_merge))

/Users/Max81007/Desktop/Python/Resource_Watch/Raster/foo.039/FertilizerConsumption_Geotiff/Foo_039_Fertilizer_TotalConsumption_Merge.tif  25230796 / 25230796.0  (100.00%)

In [13]:
os.environ["Zs3_key"] = "s3://wri-public-data/" + s3_key_merge  
os.environ["Zs3_key_inspect"] = "wri-public-data/" + s3_key_merge  

In [14]:
os.environ["Zgs_key"] = "gs://resource-watch-public/" + s3_key_merge

In [15]:
!echo %Zs3_key_inspect%
!aws s3 ls %Zs3_key_inspect%

wri-public-data/resourcewatch/raster/foo_039_Crop_Total_Nutrient_Consumption/Foo_039_Fertilizer_TotalConsumption_Merge.tif
2017-10-02 17:55:18   25230796 Foo_039_Fertilizer_TotalConsumption_Merge.tif


In [16]:
!gsutil cp %Zs3_key% %Zgs_key%

Non-MD5 etag ("c7c2816b0b8f76ab26c5de50a332f9b4-4") present for key <Key: wri-public-data,resourcewatch/raster/foo_039_Crop_Total_Nutrient_Consumption/Foo_039_Fertilizer_TotalConsumption_Merge.tif>, data integrity checks are not possible.
Copying s3://wri-public-data/resourcewatch/raster/foo_039_Crop_Total_Nutrient_Consumption/Foo_039_Fertilizer_TotalConsumption_Merge.tif [Content-Type=binary/octet-stream]...
/ [0 files][    0.0 B/ 24.1 MiB]                                                
-
- [0 files][  4.4 MiB/ 24.1 MiB]                                                
\
|
| [0 files][ 12.4 MiB/ 24.1 MiB]                                                
/
/ [0 files][ 22.2 MiB/ 24.1 MiB]                                                
-
- [1 files][ 24.1 MiB/ 24.1 MiB]                                                
\

Operation completed over 1 objects/24.1 MiB.                                     


In [17]:
os.environ["asset_id"] = "users/resourcewatch/foo_039_crop_total_nutrient_consumption"
!earthengine upload image --asset_id=%asset_id% %Zgs_key%

Started upload task with ID: ZLTEVMQR4VUQ346EL37UAYRN


In [18]:
os.environ["band_names"] = str(band_ids)

!earthengine asset set -p band_names="%band_names%" %asset_id%